## Inheritance

### Single Inheritance

    class SubClass(BaseClass):
    # ...

In [2]:
class Base:
    def __init__(self):
        print('Base initializer')
    def f(self):
        print('Base.f()')

b = Base()
b.f()

Base initializer
Base.f()


In [3]:
class Sub(Base):
    pass

s = Sub()
s.f()

In [ ]:
class Sub(Base):
    def f(self):              # Overriding method
        print('Sub.f()')

s = Sub()
s.f()        

In [ ]:
class Sub(Base):
    def __init__(self):           # Subclass initializer
        print('Sub initializer')  # Python __init__ method as any other method, override
    
    def f(self):
        print('Sub.f()')

s = Sub()
s.f()             

In [4]:
class Sub(Base):
    def __init__(self):
        super().__init__()          # calling the base initialize
        print('Sub initializer')
    
    def f(self):
        print('Sub.f()')
        
s = Sub()
s.f()                     

Base initializer
Sub initializer
Sub.f()


### Realistic inheritance example: SortedList

In [2]:
class SimpleList:
    def __init__(self, items):
        self._items = list(items)
    def add(self, item):
        self._items.append(item)
    def __getitem__(self, index):
        return self._items[index]
    def sort(self):
        self._items.sort()
    def __len__(self):
        return len(self._items)
    def __repr__(self):
        return "SimpleList({!r})".format(self._items)
    

class SortedList(SimpleList):
    def __init__(self, items=()):
        super().__init__(items)
        super().sort()
    def add(self, item):
        super().add(item)
        self.sort()
    def __repr__(self):
        return "SortedList({!r})".format(list(self))
    
sl = SortedList([4, 3, 78, 1])
sl.add(-42)
sl.add(7)
sl

SortedList([-42, 1, 3, 4, 7, 78])

## Type inspection
    isinstance()
    issubclass()

In [3]:
sl = SortedList()
isinstance(sl, SortedList)
isinstance(sl, SimpleList)

True

In [ ]:
# Checking multiple types at once
x = []
isinstance(x, (float, dict, list))

In [4]:
class IntList(SimpleList):
    def __init__(self, items=()):
        for x in items:
            self._validate(x)
        super().__init__(items)
    
    @staticmethod
    def _validate(x):
        if not isinstance(x, int):
            raise TypeError('IntList only supports integer values.')
    
    def add(self, item):
        self._validate(item)
        super().add(item)
    
    def __repr__(self):
        return "IntList({!r})".format(list(self))
    
il = IntList([1, 2, 3 ,4])
il.add(19)
il    
# il.add('5')

IntList([1, 2, 3, 4, 19])

In [ ]:
issubclass(IntList, SimpleList)
issubclass(SortedList, SimpleList)
issubclass(SortedList, IntList)

In [ ]:
#Checking the inheritance graph
class MyInt(int): pass
class MyVerySpecialInt(MyInt): pass
issubclass(MyVerySpecialInt, int)

### Multiple inheritance

Multiple inheritance simply means defining classes
with more than one direct base-class. Multiple
inheritance can lead to certain complex situations — for example, deciding what to do when
more than one base class defines a particular method 

In [1]:
class SortedIntList(IntList, SortedList):
    def __repr__(self):
        return "SortedIntList({!r})".format(list(self))

sil = SortedIntList([42, 23, 2])
sil

NameError: name 'IntList' is not defined

In [ ]:
SortedIntList([3, 2, '1'])
sil.add(-1234)
sil.add('uninteresting number')

### Automatic initializer calls

In [ ]:
class Base1:
    def __init__(self):
        print('Base1.__init__')

class Base2:
    def __init__(self):
        print('Base2.__init__')

class Sub(Base1, Base2):
    pass

s = Sub()  # Only the Base1 initializer is called

### MRO: Method Resolution Order

In Python, the method resolution order — or simply MRO — of a class is the ordering of a class’s inheritance graph used to determine which implementation to use when a method is invoked. Python uses an algorithm known as **C3 linearization** for determining MRO.
* A C3 MRO ensures that subclasses come before their base-classes
* C3 ensures that the base-class order as defined in a class definition is also preserved.
* C3 preserves the first two qualities independent of where in an inheritance graph you
  calculate the MRO.

In [12]:
SortedIntList.__mro__  # Note the order IntList and Sortedlist, change the argument order and see effect

(__main__.SortedIntList,
 __main__.IntList,
 __main__.SortedList,
 __main__.SimpleList,
 object)

In [6]:
IntList.__mro__

(__main__.IntList, __main__.SimpleList, object)

### Inconsistent MRO

In [7]:
class A: pass
class B(A): pass
class C(A): pass
class D(B, A, C): pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases A, C

## super()

    Given a method resolution order and a class C in that MRO, super() gives you
    an object which resolves methods using only the part of the MRO which comes
    after C.
### Bound and unbound proxies
Calling super return a so-called *super proxy* object. You can call any method on a super proxy, and
it will route that call to the correct method implementation if one exists. There are two high-level types of super proxies, bound and unbound. Bound proxies, as the name suggests, are bound to instances or class objects. On the other hand, unbound proxies aren’t connected to any instance, so don’t do any method dispatch themselves. Unbound
proxies are primarily an implementation detail for other Python features. Bound proxies can be bound to either classes or instances of classes. We will call these class-bound and instancebound proxies, respectively

### class-bound proxies
To create a class-bound proxy, you use this form:
        
    super(base-class, derived-class)
* Python finds the MRO for derived-class
* It then finds base-class in that MRO
* It takes everything after base-class in the MRO and finds the first class in that sequence with a method name matching the request.

In [8]:
super(SortedList, SortedIntList)

<super: __main__.SortedList, __main__.SortedIntList>

In [9]:
super(SortedList, SortedIntList).add

<function __main__.SimpleList.add(self, item)>

In [10]:
 super(SortedList, SortedIntList).add(4)

TypeError: add() missing 1 required positional argument: 'item'

In [11]:
super(SortedIntList, SortedIntList)._validate(5)
#super(SortedIntList, SortedIntList)._validate('hello')

### Instance-bound proxies

    super(class, instance-of-class)
Here, the first argument must be a class object, and the second argument must be an instance
of that class or any class derived from it.
* Python finds the MRO for the type of the second argument
* Then Python finds the location of the first argument to super() in that MRO;
* Finally, Python takes everything in the MRO after the class and uses that as the MRO
  for resolving methods.

In [15]:
sil = SortedIntList([5, 15, 10])
super(SortedList, sil)

<super: __main__.SortedList, SortedIntList([5, 10, 15])>

In [16]:
super(SortedList, sil).add(6)
sil

SortedIntList([5, 10, 15, 6])

### The no-argument versions
It turns out that you can call super() in a method with no arguments, and Python will sort
out the arguments for you.

* If you’re *in an instance method* (that is, a method which takes an instance as its first argument) and you call super() without arguments, that’s the same as calling super() with the method’s class as the first argument and self as the second.

* If you call super() without arguments *in a class method*, Python sets the arguments for you so that it’s equivalent to calling super() with the method’s class as the first argument and the classmethods first argument (that is, the “class” argument) as the second. 